In [ ]:
import pandas as pd
from igraph import Graph
import geopy

Load data

In [1]:
addrs = pd.read_csv('../data/offshore_leaks_csvs/Addresses.csv')
all_edges = pd.read_csv('../data/offshore_leaks_csvs/all_edges.csv')
entities = pd.read_csv('../data/offshore_leaks_csvs/Entities.csv', low_memory=False)
intermeds = pd.read_csv('../data/offshore_leaks_csvs/Intermediaries.csv')
officers = pd.read_csv('../data/offshore_leaks_csvs/Officers.csv')

In [18]:
entities.head()

,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,inactivation_date,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,note,valid_until,node_id,sourceID
0,ZODIAK LTD,NaN,NaN,XXX,Undetermined,Business Company Limited by Shares,Christabel Corporate Services Limited Christab...,NaN,24-AUG-2007,NaN,...,01-MAY-2009,Dead,Commonwealth Trust Limited,1427813,VGB;CYP,"Virgin Islands, British;Cyprus",NaN,The Offshore Leaks data is current through 2010,67028,Offshore Leaks
1,Zven Ltd.,NaN,NaN,XXX,Undetermined,Business Company Limited by Shares,"Alesta Consulting, S.A. INVOICE BY EMAIL ONLY",NaN,29-JAN-2007,NaN,...,01-NOV-2008,Dead,Commonwealth Trust Limited,1382756,XXX;VGB,"Not identified;Virgin Islands, British",NaN,The Offshore Leaks data is current through 2010,67243,Offshore Leaks
2,Anson 11A Ltd.,NaN,NaN,XXX,Undetermined,The GRA group of Companies for express Co.,Express Co Registration Pte Ltd 138 Cecil Stre...,NaN,09-MAY-2000,NaN,...,NaN,Liquidated,Commonwealth Trust Limited,386675,SGP;VGB,"Singapore;Virgin Islands, British",NaN,The Offshore Leaks data is current through 2010,67258,Offshore Leaks
3,GIADA LTD,NaN,NaN,XXX,Undetermined,Business Company Limited by Shares,Christabel Corporate Services Limited Christab...,NaN,14-JUN-2007,NaN,...,NaN,Transferred OUT,Commonwealth Trust Limited,1411204,CYP;VGB,"Cyprus;Virgin Islands, British",NaN,The Offshore Leaks data is current through 2010,67266,Offshore Leaks
4,Scott D. Howard,NaN,NaN,XXX,Undetermined,Sundry Account,Scott D. Howard P.O. Box 811 Brunswick ME 0401...,NaN,NaN,NaN,...,NaN,NaN,Commonwealth Trust Limited,NaN,USA;XXX,United States;Not identified,This is not an offshore entity even though it ...,The Offshore Leaks data is current through 2010,108050,Offshore Leaks


## Graph analysis

Convert `all_edges` to a graph

In [19]:
# Make Graphs
ordered_nodes = sorted(set(all_edges.node_1) | set(all_edges.node_2))

# keys are graph ids, vals are node ids
node_for_id = {x: y for x,y in enumerate(ordered_nodes)}
id_for_node = {node_for_id[id_key]: id_key for id_key in node_for_id}

g = Graph(directed=True)
g.add_vertices(len(ordered_nodes))
g.vs['node_id'] = ordered_nodes

In [20]:
merged_edges = all_edges.groupby(['node_1',
                                  'node_2']).aggregate(lambda x: set(x)).reset_index()
merged_edges['id_1'] = merged_edges.node_1.apply(lambda x: id_for_node[x])
merged_edges['id_2'] = merged_edges.node_2.apply(lambda x: id_for_node[x])

In [10]:
g.add_edges(merged_edges.loc[:, ['id_1', 'id_2']].apply(lambda x: (x[0], x[1]), 1).tolist())
g.es['types'] = merged_edges.loc[:, 'rel_type']

In [12]:
g.summary()

'IGRAPH U--- 838105 1208923 -- \n+ attr: node_id (v), types (e)'